We should have already downloaded psychopy, but if you have not, please do so.

Once psychopy is installed, download the necessary packages for creating an experiment:

In [11]:
import random, os
from psychopy import core, visual, event, gui, sound
from psychopy.hardware import keyboard
from PIL import Image
import pandas as pd

Next, we are going to set our directory variable so that we don't have to type out where our stimuli are all of the time:

In [2]:
#make sure you change this to the directory where you downloaded the stim folder!
dir = './stim/'

The first thing I like to do when running an experiment is record the subject number and any other pertinent information about the session. This can be done with a dialogue box.

In [3]:
myDlg = gui.Dlg()
myDlg.addField('Subject Number:')
myDlg.show()

['1']

To record the participant's response, you have to input that information into a variable for later use. 

In [4]:
subj_num = int(myDlg.data[0])
print(subj_num)

1


Now that you have all of the information that you need about your participant, you want to present stimuli. PsychoPy can present both visual and audio stimulus. This next step is important for presenting any type of stimulus.

This next step will open a new window, but because we are not presenting stimuli yet it will be blank.

In [5]:
#This step is important because the 'win' variable will be used a lot going forward.
#This creates a small pop-out window rather than full screen.
win = visual.Window(size= (800,  600),fullscr=None)

I recommend using the above command and not the next one for the rest of the tutorial.

This is a simple way to do full screen:

In [ ]:
 win = visual.Window(monitor="testMonitor", fullscr=True)

In [6]:
defaultKeyboard = keyboard.Keyboard(backend='event')

How to present a text stimulus:

In [9]:
#this creates a variable "intro" to be presented on the screen.
intro = visual.TextStim(win, text="You can easily present text instructions to your particpant.")
intro.draw()
win.flip()
#this command tells the program to wait one second before moving on
core.wait(1.0)
#Text can be typed out, as above, or you can create a varible to present text. We will get to that later.

How to present an image:

In [8]:
image_stim = Image.open(dir+'polarbear.jpg')
stim_size = (0.4, 0.5)
pic_draw = visual.ImageStim(win, image_stim, size = (stim_size))
pic_draw.draw()
win.update()
#this line below tells the program to wait until the space bar is pressed to exit the screen and move on
event.waitKeys(keyList=["space"])

['space']

How to present audio:

In [12]:
sound_play = sound.Sound(dir+'polarbear.aiff')
sound_play.play()
win.update()
core.wait(5.0)

Now we have the basics down, I will show you how to import data and create a loop to present multiple stimuli.

In [13]:
data = pd.read_csv(dir+'stim.csv')

In [28]:
#once the full dataset is imported, you can create a list of each stimuli
pic = data['animal_image']
audio = data['question_audio']
fact_question = data['question']
q_a = data['choice_a']
q_b = data['choice_b']
answer = data['correct_answer']

In [29]:
print(pic)

0                polarbear.jpg
1                   toucan.jpg
2                clownfish.jpg
3    woollybearcaterpillar.jpg
Name: animal_image, dtype: object


I prefer to use a while loop for experiments. You can only do this if you know how many iterations you will have. In our case, it is 4.

In [17]:
#the variable new_pic is used because now we have multiple pictures that need to be opened
#pic[n] tells the program to take one of the variables inside of pic and turn it into the new variable, new_pic
n = 0
while n != 3:
    new_pic = pic[n]
    image_stim = Image.open(dir+new_pic)
    pic_draw = visual.ImageStim(win, image_stim, size = (stim_size))

#present audio
    new_sound = audio[n]
    sound_play = sound.Sound(dir+new_sound)
    pic_draw.draw()
    sound_play.play()
    #this should show the image and play the audio at the same time
    win.update()
    event.waitKeys(keyList=["space"])
    #this increases n by 1 and allows the while loop to continue until n = 4
    n = n +1

Now that participants have completed the encoding portion, they can move on to the recall portion, which includes presenting images and text, but no audio.

In [30]:
n = 0
while n < 4:

    new_pic = pic[n]
    image_stim = Image.open(dir+new_pic)
    #this sets up the text questions that accompany the image
    q = fact_question[n]
    a1 = q_a[n]
    a2 = q_b[n]
    #this prepares the text to be put on the screen - definitely play around with postioning, etc
    question = visual.TextStim(win, text=q)
    question.pos = (0, -0.2)
    question.draw()
    answers1 = visual.TextStim(win, text="a."+a1)
    answers1.pos = (0, -0.5)
    answers1.draw()
    answers2 = visual.TextStim(win, text="b."+a2)
    answers2.pos = (0, -0.7)
    answers2.draw()
    win.update()
    keys = event.waitKeys(keyList=['a', 'b', 'escape'])

    n = n + 1

If you want to include recording responses it would look something like this:

In [31]:
output = pd.DataFrame(columns = (['Subject', 'Stim', 'Correct Answer', 'Subject Answer', 'Accuracy']))
output.loc[0, 'Subject'] = subj_num

Here is an example of how to use that to record data in our while loop:

In [35]:
#this if statement takes input from the participant response, compares it to what it should be, and provides feedback
n = 0
while n < 4:
    output.loc[n, 'Subject'] = subj_num
    if 'a' in keys:
        if 'a' == answer[n]:
            output.loc[n, 'Stim'] = new_pic
            output.loc[n, 'Correct Answer'] = answer[n]
            output.loc[n, 'Subject Answer'] = keys
            output.loc[n, 'Accuracy'] = 1
            message = visual.TextStim(win, text="That is correct!")
            message.draw()
            win.update()
            core.wait(1.0)
        if 'a' != answer[n]:
            output.loc[n, 'Stim'] = new_pic
            output.loc[n, 'Correct Answer'] = answer[n]
            output.loc[n, 'Subject Answer'] = keys
            output.loc[n, 'Accuracy'] = 0
            message = visual.TextStim(win, text="That is incorrect!")
            message.draw()
            win.update()
            core.wait(1.0)
    if 'b' in keys:
        if 'b' == answer[n]:
            output.loc[n, 'Stim'] = new_pic
            output.loc[n, 'Correct Answer'] = answer[n]
            output.loc[n, 'Subject Answer'] = keys
            output.loc[n, 'Accuracy'] = 1
            message = visual.TextStim(win, text="That is correct!")
            message.draw()
            win.update()
            core.wait(1.0)
        if 'b' != answer[n]:
            output.loc[n, 'Stim'] = new_pic
            output.loc[n, 'Correct Answer'] = answer[n]
            output.loc[n, 'Subject Answer'] = keys
            output.loc[n, 'Accuracy'] = 0
            message = visual.TextStim(win, text="That is incorrect!")
            message.draw()
            win.update()
            core.wait(1.0)
    n = n + 1

If you would like to see the participant's answers, you can either print it directly after the experiment:

In [36]:
print(output)

  Subject                       Stim Correct Answer Subject Answer Accuracy
0       1  woollybearcaterpillar.jpg              a            [b]        0
1       1  woollybearcaterpillar.jpg              b            [b]        1
2       1  woollybearcaterpillar.jpg              b            [b]        1
3       1  woollybearcaterpillar.jpg              a            [b]        0
